<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Example_2D_Navier_Stokes_Solver_(Lid_Driven_Cavity_Flow).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
nx, ny = 41, 41  # Grid points
nt = 500  # Number of time steps
dx = 2 / (nx - 1)
dy = 2 / (ny - 1)
dt = 0.001  # Time step
rho = 1  # Density
nu = 0.1  # Kinematic viscosity

# Initialize velocity and pressure fields
u = np.zeros((ny, nx))  # Velocity in x-direction
v = np.zeros((ny, nx))  # Velocity in y-direction
p = np.zeros((ny, nx))  # Pressure field
b = np.zeros((ny, nx))  # Source term for pressure Poisson equation

# Function to calculate the source term
def build_up_b(b, rho, dt, u, v, dx, dy):
    b[1:-1, 1:-1] = (rho * (1 / dt *
                   ((u[1:-1, 2:] - u[1:-1, :-2]) / (2 * dx) +
                    (v[2:, 1:-1] - v[:-2, 1:-1]) / (2 * dy)) -
                   ((u[1:-1, 2:] - u[1:-1, :-2]) / (2 * dx))**2 -
                     2 * ((u[2:, 1:-1] - u[:-2, 1:-1]) / (2 * dy) *
                          (v[1:-1, 2:] - v[1:-1, :-2]) / (2 * dx)) -
                     ((v[2:, 1:-1] - v[:-2, 1:-1]) / (2 * dy))**2))

    return b

# Function to solve the pressure Poisson equation
def pressure_poisson(p, dx, dy, b):
    pn = np.empty_like(p)
    for q in range(nt):
        pn = p.copy()
        p[1:-1, 1:-1] = (((pn[1:-1, 2:] + pn[1:-1, :-2]) * dy**2 +
                          (pn[2:, 1:-1] + pn[:-2, 1:-1]) * dx**2) /
                         (2 * (dx**2 + dy**2)) -
                         dx**2 * dy**2 / (2 * (dx**2 + dy**2)) * b[1:-1, 1:-1])

        # Boundary conditions
        p[:, -1] = p[:, -2]  # dp/dx = 0 at x = 2
        p[:, 0] = p[:, 1]    # dp/dx = 0 at x = 0
        p[-1, :] = 0         # p = 0 at y = 2
        p[0, :] = p[1, :]    # dp/dy = 0 at y = 0

    return p

# Time-stepping loop for velocity and pressure update
for n in range(nt):
    b = build_up_b(b, rho, dt, u, v, dx, dy)
    p = pressure_poisson(p, dx, dy, b)

    # Update velocities
    u[1:-1, 1:-1] = (u[1:-1, 1:-1] -
                     u[1:-1, 1:-1] * dt / dx *
                    (u[1:-1, 1:-1] - u[1:-1, :-2]) -
                     v[1:-1, 1:-1] * dt / dy *
                    (u[1:-1, 1:-1] - u[:-2, 1:-1]) -
                     dt / (2 * rho * dx) * (p[1:-1, 2:] - p[1:-1, :-2]) +
                     nu * (dt / dx**2 *
                           (u[1:-1, 2:] - 2 * u[1:-1, 1:-1] + u[1:-1, :-2]) +
                           dt / dy**2 *
                           (u[2:, 1:-1] - 2 * u[1:-1, 1:-1] + u[:-2, 1:-1])))

    v[1:-1, 1:-1] = (v[1:-1, 1:-1] -
                     u[1:-1, 1:-1] * dt / dx *
                    (v[1:-1, 1:-1] - v[1:-1, :-2]) -
                     v[1:-1, 1:-1] * dt / dy *
                    (v[1:-1, 1:-1] - v[:-2, 1:-1]) -
                     dt / (2 * rho * dy) * (p[2:, 1:-1] - p[:-2, 1:-1]) +
                     nu * (dt / dx**2 *
                           (v[1:-1, 2:] - 2 * v[1:-1, 1:-1] + v[1:-1, :-2]) +
                           dt / dy**2 *
                           (v[2:, 1:-1] - 2 * v[1:-1, 1:-1] + v[:-2, 1:-1])))

    # Boundary conditions for velocity
    u[0, :] = 0
    u[-1, :] = 0
    u[:, 0] = 0
    u[:, -1] = 1  # Lid-driven condition
    v[0, :] = 0
    v[-1, :] = 0
    v[:, 0] = 0
    v[:, -1] = 0

# Plot the velocity field
X, Y = np.meshgrid(np.linspace(0, 2, nx), np.linspace(0, 2, ny))
plt.quiver(X, Y, u, v)
plt.title("Velocity Field for 2D Lid-Driven Cavity Flow")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()